In [201]:
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [202]:
import requests
import json
from uuid import uuid4

# Complete dataspace flow

## Consts


In [203]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
api_endpoint = "https://jsonplaceholder.typicode.com/posts"

data_space_provider_participant_id = ""
data_space_consumer_participant_id = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""

json_header = {
    "Content-Type": "application/json",
}

## Participants onboarding

In [204]:
url = data_space_provider + "/api/v1/participants"
payload = {
    "participantType": "Provider",
    "participantBaseUrl": data_space_provider,
    "extraFields": {}
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["participant_id"]
data_space_provider_participant_id


'urn:uuid:c28dd861-2d35-4438-896d-7b9583796098'

In [205]:
url = data_space_provider + "/api/v1/participants"
payload = {
    "participantType": "Consumer",
    "participantBaseUrl": data_space_consumer,
    "extraFields": {
        "did": "did:web:asdnoiasdnoaisdnoisa:com"
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json["participant_id"]
data_space_consumer_participant_id

'urn:uuid:5000b412-59ff-4d6b-b489-9c0b9933a0e3'

## Create Catalog instances and metadata

In [206]:
url = data_space_provider + "/api/v1/catalogs"
payload = {
    "foaf:homepage": "My dataspace catalog",
    "dct:title": "My dataspace catalog",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]
catalog_id

'urn:uuid:8f94535d-d646-43c3-a328-8470be5814c0'

In [207]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets".format(catalog_id)
payload = {
    "dct:title": "My dataspace dataset",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
dataset_id

'urn:uuid:6ea33bb7-8ddf-4198-8165-ae8f551a3d1a'

In [208]:
url = data_space_provider + "/api/v1/catalogs/{}/data-services".format(catalog_id)
payload = {
    "dcat:endpointURL": "https://jsonplaceholder.typicode.com/posts"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_service_id = response_as_json["@id"]
data_service_id

'urn:uuid:9ecedc83-a25c-4d65-b340-1eee8d2a774a'

In [209]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets/{}/distributions".format(catalog_id, dataset_id)
payload = {
    "dcat:accessService": data_service_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
dataset_id

'urn:uuid:377f2710-7d52-4fb8-af39-0dbd0cb8f880'

## Create policies

In [210]:
url = data_space_provider + "/api/v1/datasets/{}/policies".format(dataset_id)
payload = {
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }
    ],
    "obligation": [],
    "prohibition": []
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["id"]
target_id = response_as_json["entity"]
print(json.dumps(response_as_json, indent=4))

{
    "id": "urn:uuid:f843942e-22af-41df-be60-e9bf8234df9e",
    "odrl_offer": {
        "obligation": [],
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq",
                        "rightOperand": "user"
                    }
                ]
            }
        ],
        "prohibition": []
    },
    "entity": "urn:uuid:377f2710-7d52-4fb8-af39-0dbd0cb8f880",
    "entity_type": "Dataset"
}


## Contract Negotiation

In [211]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerAddress": data_space_provider,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))


{
  "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
  "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
  "offer": {
    "@id": "urn:uuid:f843942e-22af-41df-be60-e9bf8234df9e",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:377f2710-7d52-4fb8-af39-0dbd0cb8f880"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
    "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
    "state": "REQUESTED"
  }
}


In [212]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "urn:uuid:5000b412-59ff-4d6b-b489-9c0b9933a0e3",
  "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
  "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
  "offer": {
    "@id": "urn:uuid:f843942e-22af-41df-be60-e9bf8234df9e",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:377f2710-7d52-4fb8-af39-0dbd0cb8f880"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
    "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
    "state": "OFFERED"
  }
}


In [213]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-acceptance"
payload = {
    "providerAddress": data_space_provider,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
  "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
    "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
    "state": "ACCEPTED"
  }
}


In [214]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-agreement"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))
agreement_id = response_as_json["agreement"]["@id"]

{
  "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
  "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
  "agreement": {
    "@id": "urn:uuid:92e32b5c-1ef9-4f27-924e-947b76b19470",
    "profile": null,
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "obligation": null,
    "@type": "Agreement",
    "target": "urn:uuid:377f2710-7d52-4fb8-af39-0dbd0cb8f880",
    "assigner": "urn:uuid:c28dd861-2d35-4438-896d-7b9583796098",
    "assignee": "urn:uuid:5000b412-59ff-4d6b-b489-9c0b9933a0e3",
    "timestamp": "2025-05-09 09:26:36.588814",
    "prohibition": null
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
    "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
    "state": "AGREED"
  }
}


In [215]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-verification"
payload = {
    "providerAddress": data_space_provider,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
  "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
    "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
    "state": "VERIFIED"
  }
}


In [216]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-finalization"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
  "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:19742847-b389-4d7b-baa8-938eef848b9f",
    "consumerPid": "urn:uuid:1e01a2e9-3073-4d26-a420-e38230f824af",
    "state": "FINALIZED"
  }
}


## Transfer Negotiation


In [217]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-request"
payload = {
    "providerAddress": data_space_provider,
    "agreementId": agreement_id,
    "format": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_consumer_id = response_as_json["consumerPid"]
transfer_provider_id = response_as_json["providerPid"]
callback_address = response_as_json["callbackAddress"]
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:ecdebfc6-a5af-4e57-9bd3-7e5ded9f4812",
  "consumerPid": "urn:uuid:74441508-9f4c-401c-a155-64fa4e6c7fdc",
  "agreementId": "urn:uuid:92e32b5c-1ef9-4f27-924e-947b76b19470",
  "callbackAddress": "http://127.0.0.1:1235/urn:uuid:59cef9c7-92c9-4a17-9489-ccd39f5cb881",
  "format": "http+pull",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:ecdebfc6-a5af-4e57-9bd3-7e5ded9f4812",
    "consumerPid": "urn:uuid:74441508-9f4c-401c-a155-64fa4e6c7fdc",
    "state": "REQUESTED"
  }
}


In [218]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-start"
payload = {
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
    "dataAddress": {
        "@type": "DataAddress",
        "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
        "endpoint": "http://example.com",
        "endpointProperties": [
            {
                "@type": "EndpointProperty",
                "name": "authorization",
                "value": "TOKEN-ABCDEFG"
            },
            {
                "@type": "EndpointProperty",
                "name": "authType",
                "value": "bearer"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:ecdebfc6-a5af-4e57-9bd3-7e5ded9f4812",
  "consumerPid": "urn:uuid:74441508-9f4c-401c-a155-64fa4e6c7fdc",
  "dataAddress": {
    "@type": "DataAddress",
    "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
    "endpoint": "http://example.com",
    "endpointProperties": [
      {
        "@type": "EndpointProperty",
        "name": "authorization",
        "value": "TOKEN-ABCDEFG"
      },
      {
        "@type": "EndpointProperty",
        "name": "authType",
        "value": "bearer"
      }
    ]
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:ecdebfc6-a5af-4e57-9bd3-7e5ded9f4812",
    "consumerPid": "urn:uuid:74441508-9f4c-401c-a155-64fa4e6c7fdc",
    "state": "STARTED"
  }
}


In [219]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "providerAddress": data_space_provider,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:ecdebfc6-a5af-4e57-9bd3-7e5ded9f4812",
  "consumerPid": "urn:uuid:74441508-9f4c-401c-a155-64fa4e6c7fdc",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:ecdebfc6-a5af-4e57-9bd3-7e5ded9f4812",
    "consumerPid": "urn:uuid:74441508-9f4c-401c-a155-64fa4e6c7fdc",
    "state": "SUSPENDED"
  }
}


In [220]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-start"
payload = {
    "providerAddress": data_space_provider,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:ecdebfc6-a5af-4e57-9bd3-7e5ded9f4812",
  "consumerPid": "urn:uuid:74441508-9f4c-401c-a155-64fa4e6c7fdc",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:ecdebfc6-a5af-4e57-9bd3-7e5ded9f4812",
    "consumerPid": "urn:uuid:74441508-9f4c-401c-a155-64fa4e6c7fdc",
    "state": "STARTED"
  }
}


In [221]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-completion"
payload = {
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:ecdebfc6-a5af-4e57-9bd3-7e5ded9f4812",
  "consumerPid": "urn:uuid:74441508-9f4c-401c-a155-64fa4e6c7fdc",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:ecdebfc6-a5af-4e57-9bd3-7e5ded9f4812",
    "consumerPid": "urn:uuid:74441508-9f4c-401c-a155-64fa4e6c7fdc",
    "state": "COMPLETED"
  }
}
